In [15]:
# !pip install pandas-gbq
# !pip install --upgrade google-cloud-bigquery

In [30]:
import requests
import json
from pprint import pprint
import pandas as pd
from pandas.io import gbq
import os
from dotenv import load_dotenv

data_g = None

In [42]:
def get_data():
    resource = "/resource/3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69"
    api_key = "579b464db66ec23bdd000001d16bebe4295744234023fe46120b3f81"

    url = f"https://api.data.gov.in{resource}?api-key={api_key}&format=json&limit=150"
    headers = {
        "api-key": "579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b",
        "format": "json"
    }

    response = requests.get(url, headers=headers)
    data = json.loads(response.content.decode())

    data_df = pd.DataFrame(data=data["records"])
    data_df = data_df[(data_df["pollutant_avg"] != "NA") & (data_df["pollutant_max"] != "NA") & (data_df["pollutant_min"] != "NA")]
    # data_df = data_df.astype({"pollutant_min": "int32", "pollutant_max": "int32", "pollutant_avg": "int32"})
    
    return data_df

In [43]:
data_df = get_data()
data_df.dtypes

id                object
country           object
state             object
city              object
station           object
last_update       object
pollutant_id      object
pollutant_min     object
pollutant_max     object
pollutant_avg     object
pollutant_unit    object
dtype: object

In [44]:
project_id = "steel-capsule-372509"
dataset = "realtime_aqi"
table_name = "everything"

In [45]:
data_df.to_gbq(
    destination_table=f"{dataset}.{table_name}",
    project_id=project_id,
    if_exists="replace", # fail/append/replace
)

100%|██████████| 1/1 [00:00<00:00, 5343.06it/s]


In [41]:
# from google.cloud import bigquery

# client = bigquery.Client()
# table_id = f"{project_id}.{dataset}.{table_name}"